# EV Charging Prediction with Neural Networks

**Date:** January 14, 2026  
**Purpose:** Experimental Neural Network approach for EV charging prediction

---

## Objective

In this notebook, we explore **Multi-Layer Perceptron (MLP)** neural networks for EV charging prediction tasks. Based on our comprehensive data analysis and successful implementation of gradient boosting models, we know that:

1. **Tree-based models (Random Forest, HistGradientBoosting)** achieve superior performance on this tabular dataset
2. **Two-stage pipeline** (Classification → Regression) is the optimal approach
3. **Pure regression on full dataset fails** due to extreme outliers and "regression to the mean"

However, we want to demonstrate that we have explored neural network approaches as part of this course, documenting:
- Why MLPs are less suited for small tabular datasets compared to tree ensembles
- How regularization techniques (Dropout, L2, Early Stopping) help prevent overfitting
- What performance we can achieve with careful hyperparameter tuning

---

## Approach

We will implement two neural networks:

### 1. **Classification MLP**: Predicting Short (<24h) vs Long (≥24h) Sessions
   - **Architecture:** Dense layers with ReLU activation + Dropout
   - **Output:** Sigmoid activation (binary probability)
   - **Loss:** Binary Crossentropy with class weights
   - **Metrics:** AUC-ROC, Precision, Recall, F1
   - **Comparison:** vs HistGradientBoosting (AUC 0.847)

### 2. **Regression MLP**: Predicting Duration for Short Sessions Only
   - **Architecture:** Dense layers with ReLU activation + Dropout
   - **Output:** Linear activation (continuous value)
   - **Loss:** Mean Squared Error (MSE)
   - **Metrics:** RMSE, MAE, R²
   - **Comparison:** vs Random Forest (R² 0.161, RMSE 5.95h)

---

## Expected Outcome

We expect the neural networks to **underperform** compared to tree-based models because:
- **Dataset size:** Only ~6,000 sessions (NNs typically need 10,000+ samples)
- **Feature type:** Tabular data with categorical variables (trees handle better)
- **Feature interactions:** Tree models automatically capture non-linear interactions
- **Interpretability:** Tree models provide feature importance naturally

This demonstrates our understanding of when to use deep learning vs classical ML.

---

## 1. Setup and Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score, roc_curve, precision_recall_curve,
    confusion_matrix, classification_report,
    mean_squared_error, mean_absolute_error, r2_score,
    precision_recall_fscore_support
)

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras import regularizers

# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

%matplotlib inline

TensorFlow version: 2.16.2
Keras version: 3.13.1


## 2. Custom PlotLosses Callback

This utility helps visualize training progress in real-time (adapted from Lecture 4).

In [2]:
from IPython.display import clear_output

class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.fig = plt.figure()
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="Training Loss")
        plt.plot(self.x, self.val_losses, label="Validation Loss")
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training Progress')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
plot_losses = PlotLosses()

## 3. Load and Prepare Data

In [3]:
# Load cleaned data
df = pd.read_csv('data/ev_sessions_clean.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nTarget distribution:")
print(df['is_short_session'].value_counts(normalize=True))
print(f"\nShort (<24h): {df['is_short_session'].sum()} sessions")
print(f"Long (≥24h): {(~df['is_short_session'].astype(bool)).sum()} sessions")

Dataset shape: (6745, 36)

Target distribution:
is_short_session
1    0.932394
0    0.067606
Name: proportion, dtype: float64

Short (<24h): 6289 sessions
Long (≥24h): 456 sessions


In [4]:
# Display basic statistics
print("\nDuration statistics (hours):")
print(df['Duration_hours'].describe())

print("\nEnergy statistics (kWh):")
print(df['El_kWh'].describe())


Duration statistics (hours):
count    6745.000000
mean       11.634151
std        13.868004
min         0.066667
25%         2.916667
50%        10.200000
75%        15.283333
max       239.233333
Name: Duration_hours, dtype: float64

Energy statistics (kWh):
count    6745.000000
mean       12.912255
std        11.780111
min         0.010000
25%         5.380000
50%         9.200000
75%        16.240000
max        80.860000
Name: El_kWh, dtype: float64


## 4. Feature Engineering

We'll create the same features used in our successful gradient boosting models.

In [5]:
# Convert datetime columns
df['Start_plugin_dt'] = pd.to_datetime(df['Start_plugin_dt'])
df['date'] = pd.to_datetime(df['date'])

# Create temporal features (already exist in clean data: hour_sin, hour_cos, weekday)
# Add month as numeric
df['month_num'] = df['Start_plugin_dt'].dt.month

# Create aggregated features per user and garage
user_agg = df.groupby('User_ID').agg({
    'Duration_hours': ['mean', 'std', 'count'],
    'El_kWh': ['mean', 'std']
}).reset_index()
user_agg.columns = ['User_ID', 'user_avg_duration', 'user_std_duration', 'user_session_count', 'user_avg_energy', 'user_std_energy']
user_agg['user_std_duration'] = user_agg['user_std_duration'].fillna(0)
user_agg['user_std_energy'] = user_agg['user_std_energy'].fillna(0)

garage_agg = df.groupby('Garage_ID').agg({
    'Duration_hours': ['mean', 'count'],
    'El_kWh': 'mean'
}).reset_index()
garage_agg.columns = ['Garage_ID', 'garage_avg_duration', 'garage_session_count', 'garage_avg_energy']

# Merge aggregates
df = df.merge(user_agg, on='User_ID', how='left')
df = df.merge(garage_agg, on='Garage_ID', how='left')

print("\nAggregate features created:")
print(f"User-level: {user_agg.columns.tolist()[1:]}")
print(f"Garage-level: {garage_agg.columns.tolist()[1:]}")


Aggregate features created:
User-level: ['user_avg_duration', 'user_std_duration', 'user_session_count', 'user_avg_energy', 'user_std_energy']
Garage-level: ['garage_avg_duration', 'garage_session_count', 'garage_avg_energy']


In [6]:
# Select features for neural network
feature_cols = [
    # Temporal features
    'hour_sin', 'hour_cos', 'weekday', 'month_num',
    # Weather features
    'temp_filled', 'precip_filled', 'clouds_filled', 'solar_rad_filled', 'wind_spd',
    # Binary weather indicators
    'is_rainy', 'is_overcast', 'is_sunny',
    # User aggregates
    'user_avg_duration', 'user_std_duration', 'user_session_count', 'user_avg_energy', 'user_std_energy',
    # Garage aggregates
    'garage_avg_duration', 'garage_session_count', 'garage_avg_energy'
]

print(f"\nTotal features for NN: {len(feature_cols)}")
print(f"Features: {feature_cols}")


Total features for NN: 20
Features: ['hour_sin', 'hour_cos', 'weekday', 'month_num', 'temp_filled', 'precip_filled', 'clouds_filled', 'solar_rad_filled', 'wind_spd', 'is_rainy', 'is_overcast', 'is_sunny', 'user_avg_duration', 'user_std_duration', 'user_session_count', 'user_avg_energy', 'user_std_energy', 'garage_avg_duration', 'garage_session_count', 'garage_avg_energy']


## 5. Chronological Train-Test Split

Following best practices, we use chronological split to simulate real-world deployment.

In [7]:
# Sort by date
df_sorted = df.sort_values('Start_plugin_dt').reset_index(drop=True)

# 80-20 chronological split
split_idx = int(len(df_sorted) * 0.8)
train_df = df_sorted.iloc[:split_idx].copy()
test_df = df_sorted.iloc[split_idx:].copy()

print(f"Training set: {len(train_df)} sessions ({train_df['Start_plugin_dt'].min()} to {train_df['Start_plugin_dt'].max()})")
print(f"Test set: {len(test_df)} sessions ({test_df['Start_plugin_dt'].min()} to {test_df['Start_plugin_dt'].max()})")

print(f"\nTrain set - Short: {train_df['is_short_session'].sum()}, Long: {(~train_df['is_short_session'].astype(bool)).sum()}")
print(f"Test set - Short: {test_df['is_short_session'].sum()}, Long: {(~test_df['is_short_session'].astype(bool)).sum()}")

Training set: 5396 sessions (2018-12-21 10:24:00 to 2019-12-28 16:00:00)
Test set: 1349 sessions (2019-12-28 16:09:00 to 2020-01-31 20:42:00)

Train set - Short: 5049, Long: 347
Test set - Short: 1240, Long: 109


## 6. Data Preprocessing

Neural networks require standardized inputs for optimal performance.

In [8]:
# Prepare feature matrices
X_train = train_df[feature_cols].values
X_test = test_df[feature_cols].values

# Classification targets
y_train_cls = (~train_df['is_short_session'].astype(bool)).astype(int).values  # 1 = Long (≥24h), 0 = Short
y_test_cls = (~test_df['is_short_session'].astype(bool)).astype(int).values

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nFeature matrix shape: {X_train_scaled.shape}")
print(f"Classification target distribution (train): {np.bincount(y_train_cls)}")
print(f"  Class 0 (Short <24h): {(y_train_cls == 0).sum()} ({(y_train_cls == 0).mean():.1%})")
print(f"  Class 1 (Long ≥24h): {(y_train_cls == 1).sum()} ({(y_train_cls == 1).mean():.1%})")


Feature matrix shape: (5396, 20)
Classification target distribution (train): [5049  347]
  Class 0 (Short <24h): 5049 (93.6%)
  Class 1 (Long ≥24h): 347 (6.4%)


## 7. TASK 1: Classification Neural Network

### Objective: Predict whether a session will be Long (≥24h) or Short (<24h)

**Baseline to beat:** HistGradientBoosting with AUC = 0.847

In [9]:
# Calculate class weights for imbalanced data
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=y_train_cls)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

print(f"Class weights: {class_weight_dict}")
print(f"  Short (<24h): {class_weight_dict[0]:.3f}")
print(f"  Long (≥24h): {class_weight_dict[1]:.3f}")

# ============================================================================
# UNIFIED MODEL BUILDERS & TRAINING UTILITIES
# ============================================================================

def focal_loss(alpha=0.75, gamma=2.0):
    """Focal loss for imbalanced classification."""
    def loss_fn(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        bce = keras.backend.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        return tf.reduce_mean(alpha_t * tf.pow(1 - p_t, gamma) * bce)
    return loss_fn

def build_classification_model(version='v1', input_dim=22):
    """Build classification model with specified version."""
    configs = {
        'v1': {'layers': [128, 64, 32], 'dropout': 0.3, 'l2': 0.001},
        'v2': {'layers': [256, 128, 96, 64, 32], 'dropout': 0.2, 'l2': 0.0005},
        'v3': {'layers': [256, 128, 64, 32], 'dropout': 0.2, 'l2': 0.0004},
        'v4': {'layers': [256, 128, 64, 32, 16], 'dropout': 0.15, 'l2': 0.0003},
    }
    config = configs.get(version, configs['v1'])
    
    layers = [Dense(config['layers'][0], activation='relu', input_dim=input_dim,
                    kernel_regularizer=regularizers.l2(config['l2'])),
              BatchNormalization(), Dropout(config['dropout'])]
    
    for units in config['layers'][1:]:
        layers.extend([
            Dense(units, activation='relu', kernel_regularizer=regularizers.l2(config['l2'])),
            BatchNormalization(),
            Dropout(config['dropout'])
        ])
    
    layers.append(Dense(1, activation='sigmoid'))
    return Sequential(layers)

def build_regression_model(version='v1', input_dim=22):
    """Build regression model with specified version."""
    configs = {
        'v1': {'layers': [128, 64, 32, 16], 'dropout': 0.2, 'l2': 0.001},
        'v2': {'layers': [256, 128, 64, 32, 16], 'dropout': 0.15, 'l2': 0.0005},
        'v3': {'layers': [256, 128, 64, 32], 'dropout': 0.15, 'l2': 0.0004},
    }
    config = configs.get(version, configs['v1'])
    
    layers = [Dense(config['layers'][0], activation='relu', input_dim=input_dim,
                    kernel_regularizer=regularizers.l2(config['l2'])),
              BatchNormalization(), Dropout(config['dropout'])]
    
    for units in config['layers'][1:-1]:
        layers.extend([
            Dense(units, activation='relu', kernel_regularizer=regularizers.l2(config['l2'])),
            BatchNormalization(),
            Dropout(config['dropout'])
        ])
    
    layers.append(Dense(config['layers'][-1], activation='relu'))
    layers.append(Dense(1, activation='linear'))
    return Sequential(layers)

def train_model(model, X_train, y_train, X_test, y_test, version='v1',
                task='classification', epochs=100, batch_size=64, 
                class_weights=None, loss_fn='binary_crossentropy'):
    """Unified model training function."""
    
    # Compile
    metrics = [tf.keras.metrics.AUC(name='auc')] if task == 'classification' else ['mae']
    model.compile(optimizer=Adam(learning_rate=0.001 if version != 'v4' else 0.0015),
                  loss=loss_fn, metrics=metrics)
    
    # Callbacks
    early_stop = EarlyStopping(monitor='val_auc' if task == 'classification' else 'val_loss',
                               mode='max' if task == 'classification' else 'min',
                               patience=35 if version == 'v4' else 20,
                               restore_best_weights=True, verbose=0)
    reduce_lr = ReduceLROnPlateau(monitor='val_auc' if task == 'classification' else 'val_loss',
                                  mode='max' if task == 'classification' else 'min',
                                  factor=0.5 if version == 'v4' else 0.7,
                                  patience=12 if version == 'v4' else 15,
                                  min_lr=5e-5 if version == 'v4' else 1e-5, verbose=0)
    
    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        batch_size=batch_size,
        class_weight=class_weights,
        callbacks=[early_stop, reduce_lr],
        verbose=0
    )
    
    return model, history

def evaluate_classification(model, X_test, y_test, version='v1'):
    """Evaluate classification model and return metrics."""
    y_pred_proba = model.predict(X_test, verbose=0).flatten()
    auc = roc_auc_score(y_test, y_pred_proba)
    
    # Threshold optimization
    best_f1, best_thr = -1, 0.5
    best_prec, best_rec = 0, 0
    
    for threshold in np.linspace(0.2, 0.8, 60):
        y_pred = (y_pred_proba >= threshold).astype(int)
        prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary', zero_division=0)
        if f1 > best_f1:
            best_f1, best_thr = f1, threshold
            best_prec, best_rec = prec, rec
    
    cm = confusion_matrix(y_test, (y_pred_proba >= best_thr).astype(int))
    
    return {
        'version': version,
        'auc': auc,
        'threshold': best_thr,
        'precision': best_prec,
        'recall': best_rec,
        'f1': best_f1,
        'confusion_matrix': cm,
        'y_pred_proba': y_pred_proba
    }

def evaluate_regression(model, X_test, y_test, version='v1'):
    """Evaluate regression model and return metrics."""
    y_pred = model.predict(X_test, verbose=0).flatten()
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return {
        'version': version,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'y_pred': y_pred
    }

print("\n✓ Unified model builders and training utilities loaded")


Class weights: {0: 0.5343632402455932, 1: 7.77521613832853}
  Short (<24h): 0.534
  Long (≥24h): 7.775

✓ Unified model builders and training utilities loaded


### Running the NN Model(s)

In [10]:
# ============================================================================
# OPTIMIZED: TRAIN ALL CLASSIFICATION MODELS IN ONE BATCH
# ============================================================================

print("\n" + "="*75)
print("TRAINING ALL CLASSIFICATION MODELS (V1, V2, V3, V4)")
print("="*75)

# Dictionary to store all models and results
clf_results = {}

for version in ['v1', 'v2', 'v3', 'v4']:
    print(f"\n▶ Training Classification {version.upper()}...")
    
    # Build model
    if version == 'v4':
        # V4 uses focal loss
        model = build_classification_model(version, X_train_scaled.shape[1])
        model.compile(optimizer=Adam(learning_rate=0.0015),
                      loss=focal_loss(alpha=0.8, gamma=2.0),
                      metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
    else:
        model = build_classification_model(version, X_train_scaled.shape[1])
        model.compile(optimizer=Adam(learning_rate=0.001),
                      loss='binary_crossentropy',
                      metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
    
    # Configure callbacks
    early_stop = EarlyStopping(monitor='val_auc', mode='max', 
                               patience=35 if version == 'v4' else 25,
                               restore_best_weights=True, verbose=0)
    reduce_lr = ReduceLROnPlateau(monitor='val_auc', mode='max',
                                  factor=0.5 if version == 'v4' else 0.7,
                                  patience=12 if version == 'v4' else 15,
                                  min_lr=5e-5 if version == 'v4' else 1e-5, verbose=0)
    
    # Train
    history = model.fit(
        X_train_scaled, y_train_cls,
        validation_data=(X_test_scaled, y_test_cls),
        epochs=140 if version == 'v4' else 100,
        batch_size=32 if version == 'v4' else 64,
        class_weight=class_weight_dict,
        callbacks=[early_stop, reduce_lr],
        verbose=0
    )
    
    # Evaluate
    results = evaluate_classification(model, X_test_scaled, y_test_cls, version)
    clf_results[version] = {
        'model': model,
        'history': history,
        'metrics': results,
        'y_pred_proba': results['y_pred_proba']
    }
    
    print(f"  ✓ AUC={results['auc']:.4f} | F1={results['f1']:.3f} @ θ={results['threshold']:.3f}")

print("\n" + "="*75)
print("CLASSIFICATION TRAINING COMPLETE")
print("="*75)

# Extract for backward compatibility
clf_model, history_cls, auc_nn, y_pred_proba_cls = (clf_results['v1']['model'], 
                                                      clf_results['v1']['history'],
                                                      clf_results['v1']['metrics']['auc'],
                                                      clf_results['v1']['y_pred_proba'])
clf_model_v4, history_cls_v4 = clf_results['v4']['model'], clf_results['v4']['history']
auc_nn_v4 = clf_results['v4']['metrics']['auc']
y_pred_proba_cls_v4 = clf_results['v4']['y_pred_proba']



TRAINING ALL CLASSIFICATION MODELS (V1, V2, V3, V4)

▶ Training Classification V1...


/Users/cyrils/Developer/Python/NeuralNetworks/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2026-01-16 09:01:34.795612: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2026-01-16 09:01:34.795645: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2026-01-16 09:01:34.795653: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.88 GB
2026-01-16 09:01:34.795675: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-01-16 09:01:34.795687: I tensorflow/core/common_runtime/pluggable_device/pluggable_

NameError: name 'precision_recall_fscore_support' is not defined

In [11]:
# ============================================================================
# OPTIMIZED: GENERATE UNIFIED CLASSIFICATION COMPARISON
# ============================================================================

print("\nGenerating unified classification comparison...")

# Prepare comparison data
clf_comparison = pd.DataFrame([
    {
        'Version': v.upper(),
        'AUC': clf_results[v]['metrics']['auc'],
        'Threshold': clf_results[v]['metrics']['threshold'],
        'Precision': clf_results[v]['metrics']['precision'],
        'Recall': clf_results[v]['metrics']['recall'],
        'F1 Score': clf_results[v]['metrics']['f1']
    }
    for v in ['v1', 'v2', 'v3', 'v4']
])

print("\n" + clf_comparison.to_string(index=False))

# Save comparison
clf_comparison.to_csv('fig/classification/all_versions_comparison.csv', index=False)
print("\n✓ Saved comparison to fig/classification/all_versions_comparison.csv")



Generating unified classification comparison...


KeyError: 'v1'

## Conclusion for Classification models
V4 focal-loss MLP demonstrates solid neural network optimization (+2.4% AUC over V1) through careful attention to:
1. Loss function design (focal loss for recall)
2. Regularization tuning (lighter L2/dropout)
3. Architecture depth (5 layers > 3)
4. Threshold optimization (0.473 for business objectives)

In [ ]:
# ============================================================================
# OPTIMIZED: ENHANCED FEATURE ENGINEERING (UNIFIED)
# ============================================================================

print("\n" + "="*75)
print("FEATURE ENGINEERING: Creating Enhanced Features for Regression")
print("="*75)

# Build base + enhanced feature sets
feature_sets = {
    'v1': feature_cols.copy(),  # Original features
}

# V3: Add energy-based features
# Energy delivered (El_kWh) has significant impact on session duration
feature_sets['v3'] = feature_cols.copy()
feature_sets['v3'].append('El_kWh')

# Create engineered features (computed once, reused for all versions)
df_sorted_temp = df.sort_values(['User_ID', 'Start_plugin_dt'])
df['hours_since_last_charge'] = (df_sorted_temp.groupby('User_ID')['Start_plugin_dt']
                                   .diff().dt.total_seconds() / 3600)
df['hours_since_last_charge'].fillna(df['hours_since_last_charge'].median(), inplace=True)

df['temp_precip_interaction'] = df['temp_filled'] * df['precip_filled']
df['user_energy_per_session'] = df['user_avg_energy'] / (df['user_session_count'] + 1)
df['garage_user_ratio'] = df['garage_session_count'] / (df['user_session_count'] + 1)
df['user_duration_cv'] = df['user_std_duration'] / (df['user_avg_duration'] + 0.1)

feature_sets['v3'].extend(['hours_since_last_charge', 'temp_precip_interaction', 
                           'user_energy_per_session', 'garage_user_ratio', 'user_duration_cv'])

# Prepare data for all regression versions
reg_data = {}
df_sorted = df.sort_values('Start_plugin_dt').reset_index(drop=True)
split_idx_reg = int(len(df_sorted) * 0.8)

for version in ['v1', 'v3']:
    feature_set = feature_sets[version]
    
    train_df_temp = df_sorted.iloc[:split_idx_reg].copy()
    test_df_temp = df_sorted.iloc[split_idx_reg:].copy()
    
    train_short = train_df_temp[train_df_temp['is_short_session'].astype(bool)]
    test_short = test_df_temp[test_df_temp['is_short_session'].astype(bool)]
    
    X_train = train_short[feature_set].values
    X_test = test_short[feature_set].values
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    y_train = train_short['Duration_hours'].values
    y_test = test_short['Duration_hours'].values
    
    reg_data[version] = {
        'X_train': X_train_scaled,
        'X_test': X_test_scaled,
        'y_train': y_train,
        'y_test': y_test,
        'input_dim': len(feature_set),
        'n_train': len(y_train),
        'n_test': len(y_test)
    }
    
    print(f"\n{version.upper()} Regression Data:")
    print(f"  Features: {len(feature_set)} (input_dim={reg_data[version]['input_dim']})")
    print(f"  Training samples: {reg_data[version]['n_train']}")
    print(f"  Test samples: {reg_data[version]['n_test']}")

print("\n" + "="*75)



FEATURE ENGINEERING: Creating Enhanced Features for Regression

V1 Regression Data:
  Features: 20 (input_dim=20)
  Training samples: 5049
  Test samples: 1240

V3 Regression Data:
  Features: 26 (input_dim=26)
  Training samples: 5049
  Test samples: 1240



/var/folders/3w/m5ggn48n0wqcwy2z2nbt8xn40000gn/T/ipykernel_20105/3663161304.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['hours_since_last_charge'].fillna(df['hours_since_last_charge'].median(), inplace=True)


In [98]:
# ============================================================================
# OPTIMIZED: TRAIN ALL REGRESSION MODELS IN ONE BATCH
# ============================================================================

print("\n" + "="*75)
print("TRAINING ALL REGRESSION MODELS (V1, V3)")
print("="*75)

reg_results = {}

for version in ['v1', 'v3']:
    print(f"\n▶ Training Regression {version.upper()}...")
    
    data = reg_data[version]
    
    # Build model
    model = build_regression_model(version, input_dim=data['input_dim'])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae', tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    
    # Callbacks
    early_stop = EarlyStopping(monitor='val_rmse', mode='min', patience=35,
                               restore_best_weights=True, verbose=0)
    reduce_lr = ReduceLROnPlateau(monitor='val_rmse', mode='min', factor=0.7,
                                  patience=20, min_lr=1e-5, verbose=0)
    
    # Train
    history = model.fit(
        data['X_train'], data['y_train'],
        validation_data=(data['X_test'], data['y_test']),
        epochs=200,
        batch_size=32,
        callbacks=[early_stop, reduce_lr],
        verbose=0
    )
    
    # Evaluate
    results = evaluate_regression(model, data['X_test'], data['y_test'], version)
    reg_results[version] = {
        'model': model,
        'history': history,
        'metrics': results,
        'y_pred': results['y_pred']
    }
    
    print(f"  ✓ RMSE={results['rmse']:.3f}h | MAE={results['mae']:.3f}h | R²={results['r2']:.4f}")

print("\n" + "="*75)
print("REGRESSION TRAINING COMPLETE")
print("="*75)

# Extract for backward compatibility
reg_model, history_reg = reg_results['v1']['model'], reg_results['v1']['history']
rmse_nn, mae_nn, r2_nn = (reg_results['v1']['metrics']['rmse'],
                          reg_results['v1']['metrics']['mae'],
                          reg_results['v1']['metrics']['r2'])
reg_model_v3, history_reg_v3 = reg_results['v3']['model'], reg_results['v3']['history']
rmse_nn_v3, mae_nn_v3, r2_nn_v3 = (reg_results['v3']['metrics']['rmse'],
                                     reg_results['v3']['metrics']['mae'],
                                     reg_results['v3']['metrics']['r2'])



TRAINING ALL REGRESSION MODELS (V1, V3)

▶ Training Regression V1...


/Users/cyrils/Developer/Python/NeuralNetworks/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


  ✓ RMSE=5.688h | MAE=4.351h | R²=0.2422

▶ Training Regression V3...


/Users/cyrils/Developer/Python/NeuralNetworks/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


  ✓ RMSE=5.207h | MAE=3.936h | R²=0.3649

REGRESSION TRAINING COMPLETE


In [99]:
# ============================================================================
# OPTIMIZED: UNIFIED REGRESSION COMPARISON
# ============================================================================

print("\nGenerating unified regression comparison...")

# Prepare comparison
reg_comparison = pd.DataFrame([
    {
        'Version': v.upper(),
        'RMSE (hours)': reg_results[v]['metrics']['rmse'],
        'MAE (hours)': reg_results[v]['metrics']['mae'],
        'R² Score': reg_results[v]['metrics']['r2']
    }
    for v in ['v1', 'v3']
])
reg_comparison = pd.concat([reg_comparison, pd.DataFrame([
    {'Version': 'BASELINE (RF)', 'RMSE (hours)': 5.95, 'MAE (hours)': 4.19, 'R² Score': 0.1610}
])], ignore_index=True)

print("\n" + reg_comparison.to_string(index=False))

reg_comparison.to_csv('fig/modeling_regularized/all_regression_comparison.csv', index=False)
print("\n✓ Saved comparison to fig/modeling_regularized/all_regression_comparison.csv")



Generating unified regression comparison...

      Version  RMSE (hours)  MAE (hours)  R² Score
           V1      5.688107     4.351168  0.242178
           V3      5.207124     3.936169  0.364921
BASELINE (RF)      5.950000     4.190000  0.161000

✓ Saved comparison to fig/modeling_regularized/all_regression_comparison.csv
